In [ ]:
import pandas as pd
import utils

bios = pd.read_json("../../data/congress-bios.json")
currentBios = bios[bios.congresses.apply(lambda x: 118 in x)]

...what is that at 12395?

In [ ]:
display(currentBios["profileText"][12395])

# darn it, Ben Sasse!

str12395 = (
    "a Senator from Nebraska; born in Nebraska City, Nebr., August 19, 1964; graduated"
    " Westside High School, Omaha, Nebr., 1982; B.A., biology, University of Chicago,"
    " 1986; M.B.A., marketing and finance, University of Chicago, 1991; online"
    " brokerage firm president and chief operating officer; co-owner of Chicago Cubs"
    " baseball team; unsuccessful candidate for the United States Senate in 2006;"
    " governor of Nebraska 2015-2023; appointed as a Republican to the United States"
    " Senate on January 12, 2023, to fill the vacancy caused by the resignation of"
    " Benjamin Sasse; took the oath of office on January 23, 2023."
)
currentBios.loc[12395, "profileText"] = str12395

In [ ]:
currentBios = currentBios[
    ["givenName", "familyName", "congresses", "profileText", "jobPositions"]
]

In [ ]:
currentBios.loc[:, "party"] = currentBios.jobPositions.apply(
    lambda x: x[-1]["congressAffiliation"]["partyAffiliation"][-1]["party"]["name"][0],
    # axis=1
)  # ["congress"]["congressNumber"]
currentBios.sample(5)

In [ ]:
def get_state_abbrev(text):
    for state in utils.state_list:
        if state.lower() in text.lower():
            return utils.state_dict[state]

    return None


currentBios.loc[:, "state"] = currentBios.profileText.apply(
    lambda x: get_state_abbrev(x.split(";")[0])
)

In [ ]:
currentBios.sample(5)

### Let's confirm that there are 2 senators from each State:

In [ ]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x != 2]

Okay, I assume that the NE and CA anomalies are Ben Sasse retiring and Diane
Feinstein passing away, since the two of them were both members of the 118th
Congress. Let's check on that real quick:

In [ ]:
currentBios.loc[
    (currentBios.familyName == "Sasse") | (currentBios.familyName == "Feinstein")
].index

I'm not going to drop either of them, since they were technically parts of the 118th Congress. But that still leaves the question of what's going on in VT, OK, and NC.

In [ ]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: x.state == "VT"]

Okay, looks like [Peter Welch](https://en.wikipedia.org/wiki/Peter_Welch) is missing...

In [ ]:
currentBios.loc[lambda x: (x.givenName == "Peter") & (x.familyName == "Welch")]

I guess they didn't update him in the Congressional Bioguide when he replaced Patrick Leahy? Luckily, this is an easy fix: I'll just do what I did with that massive misencoded html bio from earlier.

In [ ]:
currentBios.loc[2107, "profileText"]

They even note that he was elected as a Democrat to the Senate at the end of his bio! If I were less lazy, I'd rewrite some code to use "elected as a x to the United States Senate," but I figure so many people get appointed that the way I'm doing it now is easier.

In [ ]:
welchBio = (
    "A Senator and a Representative from Vermont; born in Springfield, Hampden County,"
    " Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.;"
    " A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of"
    " California, Berkeley, Calif., 1973; lawyer, private practice; member of the"
    " Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president"
    " pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the"
    " One Hundred First Congress in 1988; elected as a Democrat to the One Hundred"
    " Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023);"
    " was not a candidate for reelection to the House of Representatives but was"
    " elected as a Democrat to the United States Senate in 2022 for the term ending"
    " January 3, 2029."
)
currentBios.loc[2107, "profileText"] = welchBio

In [ ]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

Alright, that's VT down, now just Oklahoma and NC. My bet is that the same thing happened to Markwayne Mullin and Ted Budd.

In [ ]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: (x.state == "OK") | (x.state == "NC")]

Yep, my hunch was right.

In [ ]:
display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)
currentBios.loc[11757, "profileText"] = currentBios.loc[11757, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)

In [ ]:
display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)
currentBios.loc[9656, "profileText"] = currentBios.loc[9656, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)

In [ ]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

Alright, now we can easily flag everyone who was a senator in the 118th congress! Let's make an indicator real quick:

In [ ]:
currentBios["senator"] = (
    currentBios.profileText.str.lower().str.contains("a senator from")
    | currentBios.profileText.str.lower().str.contains("a senator and a represent")
    | currentBios.profileText.str.lower().str.contains("a senator and represent")
).astype(bool)

### Checking representatives

In [ ]:
currentBios.senator.apply(lambda x: not x).sum()

Okay, this is what I expected! There are, after all, [6 non-voting members of the House of Representatives](https://en.wikipedia.org/wiki/Non-voting_members_of_the_United_States_House_of_Representatives), and 435 = 441 - 6. I don't really care about the non-voting members, though (no offense), so I'll be dropping them (as well as making sure each state has the correct number of reps).

I might want to write a more robust script to handle Congresses before 1960 (i.e. more robust "was this state a state at the time" handling), but that isn't a problem right now.

In [ ]:
currentBios.loc[currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()]

In [ ]:
currentBios.drop(
    currentBios.loc[
        currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()
    ].index,
    inplace=True,
)

In [ ]:
reps = {
    get_state_abbrev(k): v
    for k, v in zip(utils.number_of_reps.keys(), utils.number_of_reps.values())
}

In [ ]:
# confirming that every state has the correct number of reps

[
    state
    for state in currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .index
    if currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .loc[state]
    != reps[state]
]

In [ ]:
currentBios["congressLabels"] = currentBios.apply(
    lambda row: f"{row.givenName} {row.familyName} ({row.party}-{row.state})",
    axis=1,
)

### Applying categories

Here, I put the csv into Google Sheets and did a whole lot more work on it — basically all manual, which wasn't fun. Turns out it's tough to hard-code which things ought to count as which categories. I tried some GPT-4 api calls, but I didn't end up doing too well. Good news is that, after poking around there, I can do it more efficiently here below.

The labels below are almost entirely the originals from the [New York Times piece](https://www.nytimes.com/interactive/2019/01/26/opinion/sunday/paths-to-congress.html) except that I added
- previously unsuccessful candidate for office,
- more specific master's degrees,
- previous experience in party politics.

I think I was also more generous with what counts as a medical degree, including nursing and osteopathic degrees.

You can see what I checked for each thing by examining the flags I've added for each category in in the `utils` module, though. 

In [ ]:
def splitAndTest(profileText, contains, doesntContain=[]):
    bioItems = profileText.lower().split(";")
    if doesntContain:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    if not any(dc.lower() in bioItem for dc in doesntContain):
                        return bioItem
                    else:
                        continue
        return False

    else:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    return bioItem
        return False

In [ ]:
currentBios["communityCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.communityCollege, ["board", "professor", "instructor", "president"]
    )
)

In [ ]:
currentBios["eliteCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.eliteCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
        ],
    )
)

In [ ]:
currentBios["publicCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.publicCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            # "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [ ]:
currentBios["privateCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.privateCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [ ]:
currentBios["noBA"] = currentBios.apply(
    lambda x: (
        (not x["communityCollege"])
        & (not x["eliteCollege"])
        & (not x["publicCollege"])
        & (not x["privateCollege"])
    ),
    axis=1,
)

In [ ]:
currentBios["lawSchool"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawSchool)
)

In [ ]:
currentBios["medDegree"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.medDegree)
)

In [ ]:
currentBios["doctorate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.doctorate)
)

In [ ]:
currentBios["mba"] = currentBios.profileText.apply(lambda x: splitAndTest(x, utils.mba))

In [ ]:
currentBios["otherMasters"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.otherMasters)
)

In [ ]:
currentBios["military"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.military, ["academy", "college"])
)

In [ ]:
currentBios["lawyerPrivatePractice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawyerPrivatePractice)
)
currentBios["lawyerPrivatePractice"] = [
    (
        currentBios.lawyerPrivatePractice.iloc[i]
        if currentBios.lawyerPrivatePractice.iloc[i]
        else (
            "lawyer"
            if any(
                [
                    (
                        bioItem
                        if (bioItem.strip() == "lawyer")
                        | (bioItem.strip() == "attorney")
                        else False
                    )
                    for bioItem in currentBios.profileText.iloc[i].lower().split(";")
                ]
            )
            else False
        )
    )
    for i in range(len(currentBios))
]

In [ ]:
currentBios["publicLawyerOrJudge"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.publicLawyerOrJudge, ["college counselor", "one of the managers"]
    )
)

In [ ]:
currentBios["prevUnsuccessfulCandidate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.prevUnsuccessfulCandidate)
)

In [ ]:
currentBios["partyPolitics"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.partyPolitics + ["democratic", "republican"],
        [
            "elected as a republican",
            "united states",
            "unsuccessful",
            "independent",
            "select committee on the strategic competition",
            "governor of maine",
        ],
    )
)

In [ ]:
currentBios["sciOrEng"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sciOrEng, ["b.s.", "m.s."])
)

In [ ]:
currentBios["religion"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.religion, ["b.s.", "m.s."])
)

In [ ]:
currentBios["healthcare"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.healthcare, ["b.s.", "m.s.", "m.d."])
)

In [ ]:
currentBios["blueCollarOrServiceJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.blueCollarOrServiceJob, ["academy"])
)

In [ ]:
currentBios["education"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.education, ["elected as a republican"])
)

In [ ]:
currentBios["nonprofitsAndUnions"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonprofitsAndUnions,
        [
            "credit union",
            "union, n.j.",
            "union county, n.j.",
            "union, miss.",
            "union city",
            "union theological seminary",
            "ph.d., union institute",
        ],
    )
)

In [ ]:
currentBios["sports"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sports, ["coachella valley high school"])
)

In [ ]:
currentBios["media"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.media, ["anchorage", "authority", "authorized", "speechwriter"]
    )
)

In [ ]:
currentBios["realEstate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.realEstate)
)

In [ ]:
currentBios["lawEnforcement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawEnforcement, ["attorney"])
)

In [ ]:
currentBios["farmingOrRanching"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.farmingOrRanching, ["chair, committee on agriculture", "administrator"]
    )
)

In [ ]:
currentBios["businessOrManagement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.businessOrManagement,
        [
            "campaign consultant",
            "central intelligence agency",
            "national nuclear security administration",
            "fairbanks",
            "committee on banking",
            "b.a., business administration",
            "county executive",
            "red bank, monmouth county",
            "executive office of the president",
            "committee on small business",
            "osceola",
            "nonprofit",
            "non-profit",
            "small business administration",
            "federal coordinator for gulf coast rebuilding",
            "m.b.a.",
            "m.p.a",
            "bankruptcy court",
            "business administration",
            "secretary of state",
            "government analyst",
            "california democratic state central executive committee",
            "executive director, human services",
            "jefferson county",
            "business regulation",
            "commission",
            "department of regulatory agencies",
            "nevada system of higher education",
            "republican state committee",
            "workers' compensation board",
            "afl-cio",
        ],
    )
)

In [ ]:
currentBios["lobbyingOrPacs"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lobbyingOrActivism)
)

In [ ]:
currentBios["localGov"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.localGov, ["graduated from", "b.a.", "unsuccessful candidate for"]
    )
)

In [ ]:
currentBios["stateLeg"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.stateLeg,
        ["legislative assistant", "aide", "staff", "counsel", "fellow", "unsuccessful"],
    )
)

In [ ]:
currentBios["electedFedOrStateOffice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.electedFedOrStateOffice,
        [
            "appointed",
            "assistant u.s. secretary of state",
            "inland empire regional office",
            "counsel",
            "advisor",
            "unsuccessful",
            "lost general election",
        ],
    )
)

In [ ]:
currentBios["nonElectedGovJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonElectedGovJob + ["advisor"],
        [
            "county",
            "college",
            "development advisor",
            "emergency 9-1-1 dispatcher",
            "nonprofit organization",
            "n.y. council",
            "graduated",
            "postal inspector",
            "inland empire regional office",
            "chapel hill, n.c., police department",
            "long beach, calif",
        ],
    )
)

In [ ]:
currentBios.to_json("../../data/118-processed-bios.json")